# Notebook 1 - Data Preparation & Daily KPI Construction

This notebook prepares raw airline flight data for analysis by:
- Cleaning and validating flight-level records
- Engineering delay-related features
- Aggregating daily operational KPIs
- Exporting a processed dataset for downstream analysis

The final output of this notebook is a **daily KPI dataset** used in following analytical notebooks.

## 1. Load Raw Flight Data

Raw monthly CSV files are combined into a single dataset representing the full analysis period.

In [17]:
import pandas as pd
import numpy as np
import glob 
import os

In [18]:
directory = "/Users/emiadenuga/Downloads/Portfolio Files/Airline/data/raw"
csv_files = glob.glob(os.path.join(directory, "*.csv"))

df_list = [pd.read_csv(f) for f in csv_files]

df = pd.concat(df_list, ignore_index=True)

## 2. Initial Data Validation

This step confirms:
- Dataset size and structure
- Column availability
- Presence of missing or invalid values

In [19]:
df.shape

(6847899, 12)

In [20]:
df.columns

Index(['FL_DATE', 'OP_UNIQUE_CARRIER', 'ORIGIN', 'DEST', 'CRS_DEP_TIME',
       'DEP_TIME', 'DEP_DELAY_NEW', 'CRS_ARR_TIME', 'ARR_TIME',
       'ARR_DELAY_NEW', 'CANCELLED', 'DIVERTED'],
      dtype='object')

In [21]:
df.head()


,FL_DATE,OP_UNIQUE_CARRIER,ORIGIN,DEST,CRS_DEP_TIME,DEP_TIME,DEP_DELAY_NEW,CRS_ARR_TIME,ARR_TIME,ARR_DELAY_NEW,CANCELLED,DIVERTED
0,4/1/2023 12:00:00 AM,9E,ABY,ATL,752,747.0,0.0,905,930.0,25.0,0.0,0.0
1,4/1/2023 12:00:00 AM,9E,ABY,ATL,1251,1247.0,0.0,1357,1350.0,0.0,0.0,0.0
2,4/1/2023 12:00:00 AM,9E,AEX,ATL,1003,1139.0,96.0,1250,1404.0,74.0,0.0,0.0
3,4/1/2023 12:00:00 AM,9E,AEX,ATL,1600,1551.0,0.0,1845,1828.0,0.0,0.0,0.0
4,4/1/2023 12:00:00 AM,9E,ALB,DTW,555,551.0,0.0,753,803.0,10.0,0.0,0.0


In [22]:
df.isnull().sum().sort_values(ascending=False).head(10)


ARR_DELAY_NEW        104496
ARR_TIME              89622
DEP_DELAY_NEW         84529
DEP_TIME              84469
FL_DATE                   0
OP_UNIQUE_CARRIER         0
ORIGIN                    0
DEST                      0
CRS_DEP_TIME              0
CRS_ARR_TIME              0
dtype: int64

## 3. Data Parsing & Range Check
Flights dates are converted to datetime format to enable time-based aggregation and feature engineering.

In [23]:
s = df['FL_DATE'].astype(str).str.strip()

d1 = pd.to_datetime(s, format='%Y-%m-%d', errors='coerce')
d2 = pd.to_datetime(s, format='%m/%d/%Y %I:%M:%S %p', errors='coerce')

df['FL_DATE'] = d1.fillna(d2)


df['FL_DATE'] = df['FL_DATE'].dt.strftime('%Y-%m-%d')

In [24]:
df['FL_DATE'] = pd.to_datetime(df['FL_DATE'], format='%Y-%m-%d', errors='coerce')

In [25]:
df["FL_DATE"].min(), df["FL_DATE"].max()

(Timestamp('2023-01-01 00:00:00'), Timestamp('2023-12-31 00:00:00'))

## 4. Data Cleaning

Arrival delay values may be missing for :
- Cancelled flights
- Diverted flights

Since delay-based KPIs rely on **completed flights**, rows without valid arrival delay information are removed for delay calculations while cancellations/diversion counts are preserved for reliability metrics.

In [26]:
# No row deletion of full dataset - delay handling occurs in KPI logic

## 5. Feature Engineering

### 5.1 Delay Flags / Classification

Two operational delay thresholds are defined thresholds are defined:

- **Delayed Flight** arrival delay > 15 mins
- **Severely Delayed Flight** arrival delay > 30 mins

These thresholds align with common airline on-time performance definitions.


In [27]:
df['IS_DELAYED'] = np.where(df['ARR_DELAY_NEW'] > 15,1,0)
df['IS_SEVERE_DELAYED'] = np.where(df['ARR_DELAY_NEW'] > 30,1,0)

mask_non_arrival = (df['CANCELLED'] == 1) | (df['DIVERTED'] == 1)
df.loc[mask_non_arrival, ['IS_DELAYED','IS_SEVERE_DELAYED']] = np.nan

### 5.2 Time & Route Features

Additional fields support downstream aggregation:

- **MONTH** for time-series analysis
- **ROUTE** for combining origin and destination airports

In [28]:
df['MONTH'] = df['FL_DATE'].dt.month
df['ROUTE'] = df['ORIGIN'] + '-' + df['DEST']

## 6. Daily Operational KPI Construction

Flight-level records are aggregated to a **daily route-carrier level** to create operational performance metrics.

KPIs include:
- Total flights
- Cancelled flights
- Diverted flights
- Delay rate
- Severe delay rate
- Average arrival delay (mins)
- On-time rate

These metrics form the foundation for all downstream analysis.

In [29]:
arrived = df[(df['CANCELLED'] == 0) & (df['DIVERTED'] == 0)].copy()

base = df.groupby(['FL_DATE','OP_UNIQUE_CARRIER','ORIGIN','DEST','ROUTE'], as_index=False).agg(
    FLIGHTS=('ROUTE', 'size'),
    CANCELLED_FLIGHTS=('CANCELLED', 'sum'),
    DIVERTED_FLIGHTS=('DIVERTED', 'sum')
)

arr_perf = arrived.groupby(['FL_DATE','OP_UNIQUE_CARRIER','ORIGIN','DEST','ROUTE'], as_index=False).agg(
    AVG_ARR_DELAY_MIN=('ARR_DELAY_NEW', 'mean'),
    DELAY_RATE=('IS_DELAYED', 'mean'),
    SEVERE_DELAY_RATE=('IS_SEVERE_DELAYED', 'mean'),
)

daily_kpi = base.merge(arr_perf, on=['FL_DATE','OP_UNIQUE_CARRIER','ORIGIN','DEST','ROUTE'], how='left')

daily_kpi['CANCEL_RATE'] = daily_kpi['CANCELLED_FLIGHTS'] / daily_kpi['FLIGHTS']
daily_kpi['DIVERT_RATE'] = daily_kpi['DIVERTED_FLIGHTS'] / daily_kpi['FLIGHTS']
daily_kpi['ON_TIME_RATE'] = 1 - daily_kpi['DELAY_RATE']

In [30]:
daily_kpi.head()

,FL_DATE,OP_UNIQUE_CARRIER,ORIGIN,DEST,ROUTE,FLIGHTS,CANCELLED_FLIGHTS,DIVERTED_FLIGHTS,AVG_ARR_DELAY_MIN,DELAY_RATE,SEVERE_DELAY_RATE,CANCEL_RATE,DIVERT_RATE,ON_TIME_RATE
0,2023-01-01,9E,ABE,ATL,ABE-ATL,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,2023-01-01,9E,ABY,ATL,ABY-ATL,2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2,2023-01-01,9E,AEX,ATL,AEX-ATL,2,0.0,0.0,249.5,1.0,1.0,0.0,0.0,0.0
3,2023-01-01,9E,AGS,ATL,AGS-ATL,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4,2023-01-01,9E,ALB,DTW,ALB-DTW,2,0.0,0.0,151.0,0.5,0.5,0.0,0.0,0.5


## 7. Column Selection

Only fields required for KPI analysis are retained. Irrelevant operational / identifier columns are removed to simplify the dataset and improve downstream analytical clarity.

In [31]:
final_columns = [
    'FL_DATE',
    'OP_UNIQUE_CARRIER',
    'ORIGIN',
    'DEST',
    'ROUTE',
    'FLIGHTS',
    'CANCELLED_FLIGHTS',
    'DIVERTED_FLIGHTS',
    'CANCEL_RATE',
    'DIVERT_RATE',
    'DELAY_RATE',
    'SEVERE_DELAY_RATE',
    'ON_TIME_RATE',
    'AVG_ARR_DELAY_MIN'
]
daily_kpi = daily_kpi[final_columns]

## 8. Final Dataset Validation

Confirm:
- Expected row count
- KPI ranges are realistic
- No structural issues before export

In [32]:
daily_kpi.shape

(2814071, 14)

In [33]:
daily_kpi.describe()

,FL_DATE,FLIGHTS,CANCELLED_FLIGHTS,DIVERTED_FLIGHTS,CANCEL_RATE,DIVERT_RATE,DELAY_RATE,SEVERE_DELAY_RATE,ON_TIME_RATE,AVG_ARR_DELAY_MIN
count,2814071,2.814071e+06,2.814071e+06,2.814071e+06,2.814071e+06,2.814071e+06,2.792033e+06,2.792033e+06,2.792033e+06,2.792033e+06
mean,2023-07-03 00:11:48.898105600,2.433449e+00,3.125117e-02,5.881870e-03,1.270409e-02,2.686171e-03,2.027815e-01,1.312073e-01,7.972185e-01,1.600977e+01
min,2023-01-01 00:00:00,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,2023-04-03 00:00:00,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,6.666667e-01,0.000000e+00
50%,2023-07-04 00:00:00,2.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00,1.333333e+00
75%,2023-10-02 00:00:00,3.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,3.333333e-01,0.000000e+00,1.000000e+00,1.400000e+01
max,2023-12-31 00:00:00,3.500000e+01,1.500000e+01,6.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,4.405000e+03
std,NaN,2.073475e+00,2.241538e-01,7.990354e-02,9.389466e-02,4.317082e-02,3.359904e-01,2.819509e-01,3.359904e-01,4.869247e+01


## 9. Export Processed Daily KPI Dataset

The cleaned and aggregated dataset is saved for use in:
- Notebook 2 - Exploratory & Statistical Analysis
- Notebook 3 - Dashboard Preparation & Tableau Intergration

In [ ]:
output_path = "/Users/emiadenuga/Downloads/Portfolio Files/Airline/data/processed/airline_kpi_daily.csv"
daily_kpi.to_csv(output_path, index=False)

output_path

'/Users/emiadenuga/Downloads/Portfolio Files/Airline/data/processed/airline_kpi_daily.csv'

### Notebook 1 Summary

This notebook completed:
- Raw data consildation
- Data validation & cleaning
- Delay feature engineering
- Daily KPI aggregation
- Export of analysis-ready dataset